# Merge Erddap Cruise Data and Erddap Nut Data to generate a csv file 

- merged data will need to account for when nutrients arent at a ctd location

In [86]:
from erddapy import ERDDAP
import pandas as pd
import numpy as np


In [87]:
server_url = 'http://akutan.pmel.noaa.gov:8080/erddap'


In [88]:
e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for=f'CTD final'))
print(df['Dataset ID'].values)

cruises = list(set([x.split('_')[1] for x in df['Dataset ID']]))

['CTD_ae1001_final' 'CTD_ae1401_final' 'CTD_ae1402_final'
 'CTD_ae1501_final' 'CTD_aq1201_final' 'CTD_aq1301_final'
 'CTD_aq1401l2_final' 'CTD_aq1401l3_final' 'CTD_aq1501_final'
 'CTD_aq1601_final' 'CTD_aq1801_final' 'CTD_be1201_final'
 'CTD_be1301_final' 'CTD_be1302_final' 'CTD_be1303_final'
 'CTD_cf1601_final' 'CTD_dy1006_final' 'CTD_dy1104_final'
 'CTD_dy1207_final' 'CTD_dy1208_final' 'CTD_dy1408l1_final'
 'CTD_dy1408l2_final' 'CTD_dy1408l3_final' 'CTD_dy1508_final'
 'CTD_dy1509_final' 'CTD_dy1608l1_final' 'CTD_dy1608l2_final'
 'CTD_dy1608l3_final' 'CTD_dy1609l1_final' 'CTD_dy1609l2_final'
 'CTD_dy1610_final' 'CTD_dy1704_final' 'CTD_dy1708_final'
 'CTD_dy1807l1_final' 'CTD_dy1807l2_final' 'CTD_dy1807l3_final'
 'CTD_dy1906l1_final' 'CTD_dy1906l2_final' 'CTD_dy1908_final'
 'CTD_hly1501_final' 'CTD_mb1101_final' 'CTD_me0823_final'
 'CTD_mf0610l2_final' 'CTD_mf0702_final' 'CTD_mf0706_final'
 'CTD_mf0707_final' 'CTD_mf0708_final' 'CTD_mf0711l2_final'
 'CTD_mf0802l1_final' 'CTD_mf0802l2_f

In [99]:
for cruise in cruises:
    print(f'Searching for {cruise}')
    df = pd.read_csv(e.get_search_url(response='csv', search_for=f'CTD {cruise}'))

    dfs = {}

    for dataset_id in sorted(df['Dataset ID'].values):
        if ('preliminary' in dataset_id):
            pass

        else:
            d = ERDDAP(server=server_url,
                protocol='tabledap',
                response='csv'
            )
            d.dataset_id=dataset_id

            df_m = d.to_pandas(
                        index_col='time (UTC)',
                        parse_dates=True,
                        skiprows=(1,)  # units information can be dropped.
                        )
            df_m.sort_index(inplace=True)
            df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

            dfs.update({dataset_id:df_m})

    if not ('nuts' in ",".join(sorted(df['Dataset ID'].values))):
        print(f'No nutrient data for {cruise}')
        pass
    else:
        dfs[f'CTD_{cruise}_final_nuts']['pressure'] = dfs[f'CTD_{cruise}_final_nuts']['pressure'].round()
        dfs[f'CTD_{cruise}_final']['time (UTC)'] = dfs[f'CTD_{cruise}_final'].index
        dfs[f'CTD_{cruise}_final_nuts']['time (UTC)'] = dfs[f'CTD_{cruise}_final_nuts'].index
        dfs[f'CTD_{cruise}_final']['profile_id'] = [x.split('_')[0] for x in dfs[f'CTD_{cruise}_final']['profile_id']]
        dfs[f'CTD_{cruise}_final_nuts']['profile_id'] = [x.split('_')[0] for x in dfs[f'CTD_{cruise}_final_nuts']['profile_id']]

        nut_df = pd.merge(
            dfs[f'CTD_{cruise}_final'],
            dfs[f'CTD_{cruise}_final_nuts'],
            how="left",
            on=["pressure","profile_id"],
        )

        nut_df.rename(columns={"latitude_x": "latitude",
                               'longitude_x': 'longitude',
                               'BTL_103_y': 'BTL_103',
                               'time (UTC)_x': 'time (UTC)',
                               'Station_Name_y': 'Station_Name',
                               'Water_Depth_x':'Water_Depth'}, inplace=True)
        nut_df.drop(['time (UTC)_y','latitude_y','longitude_y',
                     'BTL_103_x','Water_Depth_y','Station_Name_x'],axis=1).to_csv(f'{cruise}_merged.csv')

Searching for be1201
No nutrient data for be1201
Searching for mb1101
No nutrient data for mb1101
Searching for dy1610
No nutrient data for dy1610
Searching for dy1006
No nutrient data for dy1006
Searching for aq1501
No nutrient data for aq1501
Searching for dy1207
No nutrient data for dy1207
Searching for dy1906l3
No nutrient data for dy1906l3
Searching for mf0706
No nutrient data for mf0706
Searching for ae1501
No nutrient data for ae1501
Searching for mf0708
No nutrient data for mf0708
Searching for tn211
No nutrient data for tn211
Searching for dy1907
No nutrient data for dy1907
Searching for cf1601
No nutrient data for cf1601
Searching for aq1201
No nutrient data for aq1201
Searching for dy1906l1
No nutrient data for dy1906l1
Searching for os1701l1
No nutrient data for os1701l1
Searching for mf0810l2
No nutrient data for mf0810l2
Searching for dy1408l3
No nutrient data for dy1408l3
Searching for mf0904l2
No nutrient data for mf0904l2
Searching for aq1801
No nutrient data for aq180

In [93]:
"."

'.'